In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
import matplotlib.pyplot as plt

## Решение задачи бинарной классификации

### Логистическая регрессия

y = {-1, 1}

$b(x) = \sigma(<w,x>)$, где $\sigma(z) = \frac{1}{1 + e^{-z}}$

Поработаем с уже известным нам датасетом Titanic.

In [12]:
data = pd.read_csv('titanic.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Почистим его и оставим только интересующие нас колонки.

In [13]:
data.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
628,629,0,3,"Bostandyeff, Mr. Guentcho",male,26.0,0,0,349224,7.8958,NaN,S
331,332,0,1,"Partner, Mr. Austen",male,45.5,0,0,113043,28.5000,C124,S
229,230,0,3,"Lefebre, Miss. Mathilde",female,NaN,3,1,4133,25.4667,NaN,S
395,396,0,3,"Johansson, Mr. Erik",male,22.0,0,0,350052,7.7958,NaN,S
272,273,1,2,"Mellinger, Mrs. (Elizabeth Anne Maidment)",female,41.0,0,1,250644,19.5000,NaN,S
754,755,1,2,"Herman, Mrs. Samuel (Jane Laver)",female,48.0,1,2,220845,65.0000,NaN,S
516,517,1,2,"Lemore, Mrs. (Amelia Milley)",female,34.0,0,0,C.A. 34260,10.5000,F33,S
87,88,0,3,"Slocovski, Mr. Selman Francis",male,NaN,0,0,SOTON/OQ 392086,8.0500,NaN,S
707,708,1,1,"Calderhead, Mr. Edward Pennington",male,42.0,0,0,PC 17476,26.2875,E24,S
706,707,1,2,"Kelly, Mrs. Florence ""Fannie""",female,45.0,0,0,223596,13.5000,NaN,S


In [14]:
data.info() # Есть не только числовые признаки, но и object, есть NA

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [18]:
set(data.Sex)

{'female', 'male'}

In [19]:
def mf(s):
  return int(s == 'female')

data['Sex'] = data['Sex'].apply(mf)

In [16]:
data.drop(['Name', 'PassengerId', 'Ticket', 'Cabin'], axis=1, inplace=True) # удаляем имена и индивидуальные номера, номера кают ('Cabin') - слишком много пустых строк

In [17]:
set(data.Embarked)

{'C', 'Q', 'S', nan}

In [20]:
categ_columns = data.select_dtypes(['object']).columns
data[categ_columns] = data[categ_columns].apply (lambda x: pd.factorize(x)[0])

In [24]:
data = data.fillna(data.mean())

In [26]:
data.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.000000,1,0,7.2500,0
1,1,1,1,38.000000,1,0,71.2833,1
2,1,3,1,26.000000,0,0,7.9250,0
3,1,1,1,35.000000,1,0,53.1000,0
4,0,3,0,35.000000,0,0,8.0500,0
5,0,3,0,29.699118,0,0,8.4583,2
6,0,1,0,54.000000,0,0,51.8625,0
7,0,3,0,2.000000,3,1,21.0750,0
8,1,3,1,27.000000,0,2,11.1333,0
9,1,2,1,14.000000,1,0,30.0708,1


In [27]:
X = data.drop('Survived', axis=1)
y = data.Survived
X_train, X_test, y_train, y_test = train_test_split(X, y)

Посмотрим, сбалансированная ли у нас выборка:

In [28]:
y.value_counts()

0    549
1    342
Name: Survived, dtype: int64

Баланс не очень. Попробуем посмотреть сразу все скоры.

In [29]:
model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)
ypred_train = model.predict(X_train)
ypred_test = model.predict(X_test)
print(classification_report(ypred_train, y_train), classification_report(ypred_test, y_test))

              precision    recall  f1-score   support

           0       0.88      0.81      0.84       445
           1       0.67      0.77      0.72       223

    accuracy                           0.80       668
   macro avg       0.77      0.79      0.78       668
weighted avg       0.81      0.80      0.80       668
               precision    recall  f1-score   support

           0       0.88      0.85      0.86       141
           1       0.76      0.79      0.77        82

    accuracy                           0.83       223
   macro avg       0.82      0.82      0.82       223
weighted avg       0.83      0.83      0.83       223



In [30]:
model = SVC(kernel='linear')
model.fit(X_train, y_train)
ypred_train = model.predict(X_train)
ypred_test = model.predict(X_test)
print(classification_report(ypred_train, y_train), classification_report(ypred_test, y_test))

              precision    recall  f1-score   support

           0       0.85      0.80      0.82       436
           1       0.66      0.73      0.69       232

    accuracy                           0.78       668
   macro avg       0.75      0.76      0.76       668
weighted avg       0.78      0.78      0.78       668
               precision    recall  f1-score   support

           0       0.87      0.84      0.86       141
           1       0.74      0.78      0.76        82

    accuracy                           0.82       223
   macro avg       0.81      0.81      0.81       223
weighted avg       0.82      0.82      0.82       223



Как видим, мало отличается от работы с регрессией, синтаксис все тот же. Как sklearn понимает вообще, что у нас данные для регрессии или для классификации? А никак, мы можем спокойно применять регрессию к данным, предназначенным для классификации, и наоборот, решаем только мы сами, какой алгоритм куда больше подходит.

## Pipeline

Поработаем с датасетом про кредиты: нам нужно решить, давать кредит человеку или нет. Попробуем отмасштабировать данные и заодно собрать все в пайплайн, чтобы было удобнее.

In [3]:
data = pd.read_csv('loan_sanction_train.csv')
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


Поработаем с признаками и дропнем пропуски.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [5]:
data = data.dropna()

In [6]:
data.drop('Loan_ID', axis=1, inplace=True)

In [49]:
set(data.Property_Area)

{'Rural', 'Semiurban', 'Urban'}

In [64]:
set(data.Dependents)

{'0', '1', '2', '3+'}

In [51]:
def mf(s):
  return int(s == 'Female')

data['Gender'] = data['Gender'].apply(mf)

In [59]:
def marriage(s):
  return int(s == 'Yes')

data['Married'] = data['Married'].apply(marriage)

In [60]:
def educ(s):
  return int(s == 'Graduate')

data['Education'] = data['Education'].apply(educ)

In [61]:
def self_emp(s):
  return int(s == 'Yes')

data['Self_Employed'] = data['Self_Employed'].apply(self_emp)

In [62]:
def status(s):
  return int(s == 'Y')

data['Loan_Status'] = data['Loan_Status'].apply(status)

In [66]:
def depends(d):
  if d == '3+':
    return 3
  else:
    return int(d)

data['Dependents'] = data['Dependents'].apply(depends)

In [68]:
data = pd.get_dummies(data)

Разделим на трейн и тест.

In [70]:
X = data.drop('Loan_Status', axis=1)
y = data.Loan_Status
X_train, X_test, y_train, y_test = train_test_split(X, y)

Соберем пайплайн: комбайн, который будет внутри себя сразу гонять и масштабирование, и модель

In [71]:
pipe = Pipeline([('scaler', StandardScaler()), ('model', LogisticRegression())])

Альтернативный вариант:

pipe = make_pipeline(StandardScaler(), LogisticRegression())

В чем между ними разница? Во-первых, второе - упрощенный синтаксис, вы не прописываете вручную ярлычки для своих шагов пайплайна. Во-вторых, получается, эти ярлычки приписываются автоматически (по правилу: название класса строчными буквами, например, у StandardScaler автоматически будет ярлычок standardscaler). Это сакральное знание пригодится, когда будем гридсерчить параметры.

In [72]:
pipe.fit(X_train, y_train)
ypred_train = pipe.predict(X_train)
ypred_test = pipe.predict(X_test)
print(classification_report(ypred_train, y_train), classification_report(ypred_test, y_test))

              precision    recall  f1-score   support

           0       0.42      0.89      0.57        57
           1       0.97      0.77      0.86       303

    accuracy                           0.79       360
   macro avg       0.70      0.83      0.72       360
weighted avg       0.89      0.79      0.81       360
               precision    recall  f1-score   support

           0       0.52      0.88      0.65        16
           1       0.98      0.88      0.92       104

    accuracy                           0.88       120
   macro avg       0.75      0.88      0.79       120
weighted avg       0.92      0.88      0.89       120



### Несбалансированные классы

Давайте порешаем еще одну финансовую задачку: будем предсказывать, возьмет клиент банка кредит или нет.

In [32]:
data = pd.read_csv('bank-additional-full.csv', sep=';')
data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


Для начала просто дропнем все нечисловые характеристики.

In [33]:
X = data.select_dtypes(include=np.number)
y = data.y

Проверим распределение классов:

In [34]:
y.value_counts()

no     36548
yes     4640
Name: y, dtype: int64

Обучите обычную логистическую регрессию на этом датасете и выведите classification report.

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y) # параметр stratify делает подвыборки с равномерным (по возможности) распределением классов

model = LogisticRegression(solver='liblinear')
model.fit(X_train, y_train)
ypred_train = model.predict(X_train)
ypred_test = model.predict(X_test)
print(classification_report(ypred_train, y_train), classification_report(ypred_test, y_test))

# Выводы: видимо, несбалансированность классов дает большую разницу в результатах для классов 0 и 1,
# разница в precision заметнее, чем в recall

              precision    recall  f1-score   support

          no       0.98      0.93      0.95     28847
         yes       0.39      0.67      0.49      2044

    accuracy                           0.91     30891
   macro avg       0.68      0.80      0.72     30891
weighted avg       0.94      0.91      0.92     30891
               precision    recall  f1-score   support

          no       0.98      0.93      0.95      9617
         yes       0.39      0.66      0.49       680

    accuracy                           0.91     10297
   macro avg       0.68      0.80      0.72     10297
weighted avg       0.94      0.91      0.92     10297



Какие выводы можете сделать на основании метрик?

А теперь давайте применим особую магию с class_weight.

In [36]:
model = LogisticRegression(solver='liblinear', class_weight='balanced')
model.fit(X_train, y_train)
ypred_train = model.predict(X_train)
ypred_test = model.predict(X_test)
print(classification_report(ypred_train, y_train), classification_report(ypred_test, y_test))

# Пробуем сбалансировать классы - показатели для классов 0 и 1 стали ближе.

              precision    recall  f1-score   support

          no       0.85      0.98      0.91     23808
         yes       0.85      0.42      0.56      7083

    accuracy                           0.85     30891
   macro avg       0.85      0.70      0.74     30891
weighted avg       0.85      0.85      0.83     30891
               precision    recall  f1-score   support

          no       0.85      0.98      0.91      7975
         yes       0.84      0.42      0.56      2322

    accuracy                           0.85     10297
   macro avg       0.85      0.70      0.74     10297
weighted avg       0.85      0.85      0.83     10297

